In [1]:

import json
from bs4 import BeautifulSoup

In [17]:
f = open('TRECcovid/corpus_small.jsonl').readlines()

In [26]:
corpus = [json.loads(i) for i in json.loads(json.dumps(f))]

In [30]:
corpus[0]['text']

'BACKGROUND: There are sparse data on whether non-pharmaceutical interventions can reduce the spread of influenza. We implemented a study of the feasibility and efficacy of face masks and hand hygiene to reduce influenza transmission among Hong Kong household members. METHODOLOGY/PRINCIPAL FINDINGS: We conducted a cluster randomized controlled trial of households (composed of at least 3 members) where an index subject presented with influenza-like-illness of <48 hours duration. After influenza was confirmed in an index case by the QuickVue Influenza A+B rapid test, the household of the index subject was randomized to 1) control or 2) surgical face masks or 3) hand hygiene. Households were visited within 36 hours, and 3, 6 and 9 days later. Nose and throat swabs were collected from index subjects and all household contacts at each home visit and tested by viral culture. The primary outcome measure was laboratory culture confirmed influenza in a household contact; the secondary outcome w